In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import warnings
import ipdb
import dan_utils

warnings.filterwarnings("ignore")

In [2]:
from statsmodels.graphics.tsaplots import plot_acf
from statsmodels.graphics.tsaplots import plot_pacf
from statsmodels.tsa.stattools import adfuller as ADF
from statsmodels.stats.diagnostic import acorr_ljungbox
from statsmodels.tsa.arima_model import ARIMA

In [3]:
def base_arima(data, pre_step):
    # data should be 1-D DataFrame
    D_data=data.diff(periods=1).dropna()
    data = np.array(data)

    model=ARIMA(data,(1,1,1)).fit()
    forecast=model.forecast(pre_step)
    return (forecast[0])

In [4]:
randseed = 25
dan_utils.setup_seed(randseed)
res = 11

v = pd.read_csv('../data/q_20_aggragated.csv')
v = v.rename(columns={'Unnamed: 0': 'id'})
det_with_class = pd.read_csv('../res/%i_res%i_id_402_withclass.csv'%(randseed, res), index_col=0)

v['class_i'] = ''
for i in range(len(v)):
    v.loc[i, 'class_i'] = det_with_class[det_with_class['id']==v.loc[i, 'id']].iloc[0, 5]  # 5 stands for 'class_i'

num_class = det_with_class['class_i'].drop_duplicates().size

v_class = []
for i in range(num_class):
    v_class.append(v[v['class_i']==i])

print('There are %i class(es)'%num_class)

dist_mat = pd.read_csv('../data/dist_mat.csv', index_col=0)
id_info = pd.read_csv('../data/id2000.csv', index_col=0)
dist_mat.index = id_info['id2']
dist_mat.columns = id_info['id2']
for i in range(len(dist_mat)):
    for j in range(len(dist_mat)):
        if i==j:
            dist_mat.iloc[i, j] = 0

near_id = pd.DataFrame(np.argsort(np.array(dist_mat)), index = id_info['id2'], columns = id_info['id2'])

seg = pd.read_csv('../data/segement.csv', header=None)
num_dets = 25

det_list_class = []
for i in range(num_class):
    det_list_class_temp, v_class_temp = dan_utils.get_class_with_node(seg, v_class[i])
    det_list_class.append(det_list_class_temp)
    v_class_temp = v_class_temp[v_class_temp['id'].isin(det_list_class_temp[:num_dets])]
    v_class[i] = v_class_temp
    
near_road_set = []
for i in range(num_class):
    near_road_set.append(dan_utils.rds_mat(dist_mat, det_list_class[i][:num_dets], seg))

There are 5 class(es)


In [5]:
# ind, class
# 0  , blue
# 1  , green
# 2  , yellow  <--
# 3  , black   <--
# 4  , red     <--
class_color_set = ['b', 'g', 'y', 'black', 'r']
class_i = 4

# v_class[4].iloc[:, 2:-1]

In [24]:
data = np.array(v_class[4].iloc[:, 2:-1])
window = 100
pred_num = 6

pred_mat_all = []
label_mat_all = []
for i in range(data.shape[0]):  # iterate over detectors
    pred_mat = []
    label_mat = []
    for j in range(data.shape[1] - window - pred_num):
        data_temp = data[i, j:j+window]
        label = data[i, j:j+window+pred_num]
        pred = base_arima(pd.DataFrame(data_temp), pred_num)
        pred_mat.append(pred)
        label_mat.append(label)
    pred_mat_all.append(np.array(pred_mat))
    label_mat_all.append(np.array(label_mat))

C:\Users\10169\anaconda3\envs\dan_traff\lib\site-packages\statsmodels\base\model.py:547: HessianInversionWarning: Inverting hessian failed, no bse or cov_params available
  warnings.warn('Inverting hessian failed, no bse or cov_params '
C:\Users\10169\anaconda3\envs\dan_traff\lib\site-packages\statsmodels\base\model.py:547: HessianInversionWarning: Inverting hessian failed, no bse or cov_params available
  warnings.warn('Inverting hessian failed, no bse or cov_params '
C:\Users\10169\anaconda3\envs\dan_traff\lib\site-packages\statsmodels\base\model.py:547: HessianInversionWarning: Inverting hessian failed, no bse or cov_params available
  warnings.warn('Inverting hessian failed, no bse or cov_params '
C:\Users\10169\anaconda3\envs\dan_traff\lib\site-packages\statsmodels\base\model.py:566: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  warnings.warn("Maximum Likelihood optimization failed to "
C:\Users\10169\anaconda3\envs\dan_traff\lib\site-

ValueError: The computed initial MA coefficients are not invertible
You should induce invertibility, choose a different model order, or you can
pass your own start_params.